In [2]:
#Import the packages
import ROOT as r
import numpy as np
import math

# Part 0. The datasets
 
 1. **Z to $\mu\mu$ simultion**

We will begin by looking at simulation data corresponding to a Z boson decaying into two muons, which we will be comparing with real CMS data later on after getting familiar with the software.

These simulated datasets correspond to the collision data collected by the CMS experiment in 2012.

First, let's understand what we are looking at.

The dataset name is: 

`/DYToMuMu_M-20_CT10_8TeV-powheg-pythia6/Summer12_DR53X-PU_S10_START53_V19-v1/AODSIM`

The dataset name follows a specific convention to provide a self-explanatory description of the dataset.

It follows a pattern like this:

`/Primary dataset/Condition/Format/`

 - The `Primary dataset` tells you about the process for a simulation dataset, and the data taking period and trigger stream that the data was collected if it is data.

 - The `Condition` part tells you what reconstruction condition has been applied to the dataset.

 - Lastly, the `Format` part tells you the dataformat. We will work with the simplified version of `MINIAOD` format, called `NanoAOD`, which I have prepared for you in advance.**

The dataset is downloaded from here. The files `dy_mc.root`/`dy_mc_small.root` is a tiny fraction of the whole dataset, as we do not need the whole 14TB of events.  
http://opendata.cern.ch/record/7741

 2. **Data - Double Muon data**

We will be using proton-proton collision data at $\sqrt{s}=8 TeV$ taken during 2012 by the CMS.

Of course, we will not be using all of them, the dataset name 

`/DoubleMuParked/Run2012B-22Jan2013-v1/AOD`

tells you with subset of data we will be using.

Here `DoubleMuParked` means the data collected was triggered by a stream of High Level Trigger(HLT) trigger paths that targets muon, such as
```
HLT_DoubleMu5_IsoMu5
```
which means it requires two muons with at least 5 GeV and one of them is isolated from other particles.

Like the simulation data, a skimmed version have been made for you: `Run2012B_DoubleMuParked_nanoaod.root`

More details can be found in this record:
http://opendata.cern.ch/record/6004


<br>
<br>
<br>


** In case you are curious, the conversion is done also with cms open software here:

https://github.com/cms-opendata-analyses/AOD2NanoAODOutreachTool

# Part 1 Exploring ROOT files with pyROOT

In this section, we will go over some of the basic operations to explore a root file, working with the small simulation file first.


In [ ]:
#Opening a ROOT file
f = r.TFile("dy_mc_small.root")
# list the content of the file
f.ls()
dir = f.GetDirectory("aod2nanoaod")
dir.ls()

### TTree
TTree is the main object that holds the data. 

In general, all the objects in a TFile can be obtained with the `Get` Method

In [22]:
t = f.Get("aod2nanoaod/Events")

In [ ]:
## See how many events are there in this tree
nEntries = t.GetEntries()
print(nEntries)

### TTree.Show()
 There a several methods to see what's inside a tree

`Show(n)` prints out the n-th event in the tree

In [ ]:
t.Show(0)

### TTree.Print()

`Print()` gives you more information about the branches, esp dataformat. 

Here, `run/I` tells you that the branch `run` is of type integer

whereas, `PV_x/F` means PV_x is a float

Finally, `Muon_pt[nMuon]/F ` is a array of floats

In [ ]:
t.Print()

### Question: Why do we need a different data format for the Muon_pt branch?

### TTree::Draw

Another very handy way to see the data is the `Draw` method.

This is an internal method to loop over all the events on a branch (very efficiently!),

and fill an histogram from the branch content

To visualize the histogram, we first need to create a Canvas.


In [ ]:
c = r.TCanvas('c1','c1',800,600)

For example, we can look at the branch called `nMuon`

In [ ]:
t.Draw('nMuon')
c.Draw()

### Question: Can you guess what does this histogram represents?

The branch names usually contains good description of what does it contain.

Similarly, we can explore another branch called `PV_npvs` as well

In [ ]:
t.Draw("PV_npvs")
c.Draw()

### Question: Can you guess what `PV_npvs` represents then?

### Looping a TTree with a histogram

`TTree.Draw()` can only offer a simple plotting, for more custom analysis, we can go through the `TTree` event-by-event by building a for-loop.

Let's use a simple for-loop to re-create the `nMuon` histogram.

 - Learn how to create a histogram
 - Learn how to write a loop structure for a TTree

### Defining your own 1D histogram

`ROOT` has a very powerful library of histogram class called `TH1` with extensive functionalities.

You can construct a very simple 1D histogram like this:

`h = r.TH1F("name","title", nBin, x_low, x_hi)` 

 - `TH1F` is a sub-class of `TH1`, using a `float` to store the data of each bin of the histogram
 - `name` is the name of the histogram object, it is used to identify the histogram object (h, in this case) in a file
 - `title` is the title to be displayed in plotting
 - `nBin` is the number of bins 
 - `x_low,x_hi` are the lower and upper range of the 1D histogram respectively.
 
 
Further documentation about `TH1` can be found here:
https://root.cern.ch/doc/master/classTH1.html

### Let's define our first histogram

In [ ]:
h_nMuon = r.TH1F('nMuon','nMuon',10,0,10)
h_nMuon.Draw()
c.Update()
c.Draw()

### Filling a histogram

Say, if you have an event with 4 muons in it. You will want to put 1 event in the 4-th bin.

In [ ]:
h_nMuon.Fill(4)
c.Update()
c.Draw()

In [12]:
## now let's clear the bin content, by setting the 5th bin to 0
h_nMuon.SetBinContent(5,0)

### Looping a TTree

By default, all the branches are loaded as the methods of the TTree object.

For example, you can simply access the `nMuon` branch like this:

`nMuon = t.nMuon`

**It will return the value of the tree is currently located

To access different events, we can do:

`t.GetEntry(i)`

This will set the memeory address to load the i-th entry in the Tree.

Combining both, a TTree can be looped over with the simple loop structure like this:

```
for i in range(0,t.GetEntries()):
    t.GetEntry(i)
    nMuon = t.nMuon
    h_nMuon.Fill(nMuon)
```

In [13]:
for i in range(0,t.GetEntries()):
    t.GetEntry(i)
    nMuon = t.nMuon
    h_nMuon.Fill(nMuon)

In [ ]:
h_nMuon.Draw()
c.Update()
c.Draw()

# Part 3. Plotting the di-muon mass spectrum

Now we have all the ingredients to plot the di-muon mass spectrum.

First, we will need a histogram.

In [23]:
h_dimuon_mass = r.TH1F('DiMuon_mass','DiMuon_mass',200,0,200)
h_dimuon_mass.Sumw2()

You may find the following formulas useful:

Coordinate transformation between pesudeo-rapidity and cartesian coordinates:

$
p_x = p_T cos(\phi)\\
p_y = p_T sin(\phi)\\
p_z = p_T sinh(\eta)
$

Vector additions and Energy-momentum relations:

$
E^2 = |\vec{p}|^2 + m^2\\
\vec{p_1}+\vec{p_2} =  (p1_x+p2_x) \hat{x}+ (p1_y+p2_y) \hat{y}+ (p1_z+p2_z) \hat{z}\\
$

In [39]:
## write a function that computes the invariant mass given the event
## pt, phi, eta, mass are the arrays of the muons in each event
##
def invmass(pt,phi,eta,mass):
    m = 0
    ####################
    ## YOUR CODE HERE ##
    ####################
    return m

In [38]:
## write a function that select 
def selectMuons(muons):
    select = False

    # Minimum selection for di-muon
    if muons['n']==2: select = True
    return select

In [ ]:
for i in range(0,nEntries):
    t.GetEntry(i)
    nMuon   = t.nMuon
    pt      = np.array(t.Muon_pt)          # note: this is an array
    phi     = np.array(t.Muon_phi)         # note: this is an array
    eta     = np.array(t.Muon_eta)         # note: this is an array 
    mass    = np.array(t.Muon_mass)        # note: this is an array
    isTight = np.array(t.Muon_tightId)     # note: this is an array
    charge  = np.array(t.Muon_charge)      # note: this is an array
    muons   = {
        'n':nMuon,
        'pt':pt,
        'phi':phi,
        'eta':eta,
        'isTight':isTight,
        'charge':charge,
    }
    if selectMuons(muons):
        h_dimuon_mass.Fill(invmass(pt,phi,eta,mass))
        
h_dimuon_mass.Draw('hist')
h_dimuon_mass.GetXaxis().SetTitle('m_{#mu#mu}[GeV]')
c.Update()
c.SetLogy(1)
c.Draw()

### RESET the histogram if you want to try again!

In [99]:
for i in range(0,201): h_dimuon_mass.SetBinContent(i,0)

# Part 3 Extracting the Z-boson mass

Let's try to extract the Z-boson mass by fitting the peak with a Gaussian function.

Mathematical functions in ROOT can be constructed with the class `TF1`*
Gaussian function is one of the built-in function in ROOT with the following form:

$f(x) = [0] exp\left(-\frac{1}{2}((x-[1])/[2]\right)^2$

Here `[0],[1],[2]` are the parameters of the function, corresponding to the `amplitude`, `mean` and `width` of the Gaussian function.

Let's create a gaussian function, initialize it with some values, and draw it.

* Full documentation of the `TF1` class
https://root.cern.ch/doc/master/classTF1.html

In [ ]:
## Here, 'f1' is the name of the object, 
## 'gaus' is a special string that corresponds to the gaussian formula above
## 85 and 95 is the range of the function.
## i.e. we are insterested only in that mass window.
gaus = r.TF1('f1','gaus',75,105)
gaus.SetParameters(1,90,10)
gaus.Draw()
c.SetLogy(0)
c.Draw()

`TH1` has basic fit function implemented.

The default option is the least square fit, 

$
\chi^2 = \sum \left((y_i - f(x_i) )/e_i \right)^2
$

The option `R` tells the fit to restrict to the range defined by the function

The options `S` simply return the fit result as an object instead of just print-outs.

The full options can be found here:

https://root.cern.ch/doc/master/classTH1.html#a63eb028df86bc86c8e20c989eb23fb2a

In [ ]:
fitResult = h_dimuon_mass.Fit(gaus,'RS')

In [ ]:
h_dimuon_mass.Draw()
c.Draw()

## Questions:
- How does the width of the Gaussian (calculate the full width at half maximum from sigma) compares with the known intrinsic width of the Z boson?


## Part 4a Quality of the fit

In [ ]:
chi2 = gaus.GetChisquare()
NDF  = gaus.GetNDF()
print('Checking fit quality:')
print("chi2 = %.3f\nNDF = %.3f\nchi2/NDF = %.3f"% (chi2,NDF, chi2/NDF))

# Part 4 Take-home exercise

# Part 4a Knowing more about the muons

Now we have the basic tools to explore a ROOT file and analysis. Let's learn deeper about the muons!

Here are some questions to think about:

- plot muon pT for the leading, subleading, and third-leading muon (when it's there) and explain the features seen in the plot;
- plot muon eta for the leading and subleading muons and explain the feature seen in the plot;
- plot isTight flag for the three muons;
- plot the muon multiplicity for isTight muons;

These questions can be solved in a similar way as the `nMuon` example, with the addition of some control statements during the histogram filling.

### Hint: all the `Muon_*` branch has already been sorted in decending order of `pt` 
i.e `Muon_pt[0]` corresponds to the pt of the leading muon, `Muon_eta[0]` would be the $\eta$ of the same muon


To help you get started, let's define some histograms for you.

In [14]:
## Define the histograms
h_mu0_pt      = r.TH1F('h_mu0_pt'     ,'Leading muon pt',200,0,200)
h_mu0_eta     = r.TH1F('h_mu0_eta'    ,'Leading muon eta',100,-5,5)
h_mu0_isTight = r.TH1F('h_mu0_isTight','Leading muon isTight',2,0,2)

h_mu1_pt      = r.TH1F('h_mu1_pt'  ,'Sub-leading muon pt',200,0,200)
h_mu1_eta     = r.TH1F('h_mu1_eta' ,'Sub-leading muon eta',100,-5,5)
h_mu1_isTight = r.TH1F('h_mu1_isTight','Sub-leading muon isTight',2,0,2)

h_mu2_pt      = r.TH1F('h_mu2_pt','Third-Leading muon pt',200,0,200)
h_mu2_isTight = r.TH1F('h_mu2_isTight','Third-leading muon isTight',2,0,2)

h_nTightMuon  = r.TH1F('h_nTightMuon','Number of tight muons',10,0,10)

#collect the histograms into a dic
histograms = {
        'h_mu0_pt'     :h_mu0_pt,
        'h_mu0_eta'    :h_mu0_eta,
        'h_mu0_isTight':h_mu0_isTight,
        'h_mu1_pt'     :h_mu1_pt,
        'h_mu1_eta'    :h_mu1_eta,
        'h_mu1_isTight':h_mu1_isTight,
        'h_mu2_pt'     :h_mu2_pt,
        'h_mu2_isTight':h_mu2_isTight,
        'h_nTightMuon' :h_nTightMuon,
}

In [16]:
for i in range(0,nEntries):
    t.GetEntry(i)
    nMuon   = t.nMuon
    pt      = np.array(t.Muon_pt)          # note: this is an array
    phi     = np.array(t.Muon_phi)         # note: this is an array
    eta     = np.array(t.Muon_eta)         # note: this is an array 
    mass    = np.array(t.Muon_mass)        # note: this is an array
    isTight = np.array(t.Muon_tightId)     # note: this is an array
    charge  = np.array(t.Muon_charge)      # note: this is an array
    muons   = {
        'n':nMuon,
        'pt':pt,
        'phi':phi,
        'eta':eta,
        'isTight':isTight,
        'charge':charge,
    }
    #### YOUR CODE HERE ###

### Drawing the muon pT histograms

You can execute the following cells to check the histograms

In [ ]:
h_mu0_pt.SetLineColor(r.kGreen)   ## r.kRed is a global constant stored in ROOT, corresponding to the color code
h_mu1_pt.SetLineColor(r.kRed)     ## r.kRed is a global constant stored in ROOT, corresponding to the color code
h_mu2_pt.SetLineColor(r.kBlue)
h_mu0_pt.Draw()
h_mu1_pt.Draw('same')             ## the "same" option overlays the histograms on the same canvas
h_mu2_pt.Draw('same')
h_mu0_pt.GetXaxis().SetTitle('pT[GeV]')
c.Update()
c.Draw()

### Question: What do you observe? Can you understand the distribution?

YOUR ANS HERE

### Drawing the muon eta histograms

In [ ]:
h_mu0_eta.Draw()
h_mu0_eta.SetLineColor(r.kRed)     ## r.kRed is a global constant stored in ROOT, corresponding to the color code
h_mu1_eta.SetLineColor(r.kBlue)
h_mu1_eta.Draw('same')             ## the "same" option overlays the histograms on the same canvas
h_mu0_eta.GetXaxis().SetTitle('#eta')
c.Update()
c.Draw()

### Drawing isTight histograms

In [ ]:
h_mu0_isTight.Draw()
h_mu0_isTight.SetLineColor(r.kRed)     ## r.kRed is a global constant stored in ROOT, corresponding to the color code
h_mu1_isTight.SetLineColor(r.kBlue)
h_mu2_isTight.SetLineColor(r.kGreen)
h_mu1_isTight.Draw('same')             ## the "same" option overlays the histograms on the same canvas
h_mu2_isTight.Draw('same')             ## the "same" option overlays the histograms on the same canvas
h_mu0_isTight.GetXaxis().SetTitle('isTight')
c.Update()
c.SetLogy(1)
c.Draw()

In [ ]:
h_nTightMuon.Draw()
c.SetLogy(0)
c.Update()
c.Draw()

### Remember to reset the histograms if you want to re-fill them!

In [22]:
for hname,h in histograms.items():
    for b in range(0,h.GetNbinsX()):
        h.SetBinContent(b,0)

# Part 4b Di-muon selections:
 - Can we increase the number of events by using events with exactly two isTight muons, not exactly two muons?
   - Try modifying the `selectMuon` function to test your ideas!
 - histogram the product of electric charges for the events with two tight muons; How often do we have same-sign muons?

# Part 4c Z-boson decays

Now that you have observe a Z boson, you do more in-depth analysis of those events coming from a Z boson decay.

- For events within an invariant mass window near the Z peak, plot pT spectrum of the Z boson.
- For these events, plot a correlation (2D histogram) of the opening azimuthal angle (phi) between the two muons and the pT of the Z boson.

You can use the following histograms:

In [29]:
h_pT_z    = r.TH1F('pT_z','pT of Z boson',100,0,100)
h_pTz_phi = r.TH2F('pTz_phi','pT of Z boson v.s. #Delta#phi(#mu#mu)',100,0,100,50,0,math.pi)

In [35]:
# We need to be careful when we calculate azimuthal angle difference, since computers do not know 2pi+phi = phi
# so we need to have a function that returns deltaPhi to within 0 to pi
def deltaPhi(phi1, phi2):
    x = np.abs(phi1 - phi2)
    if x >=  math.pi: 
        x = 2.*math.pi - x
    return x

In [31]:
for i in range(0,nEntries):
    t.GetEntry(i)
    nMuon   = t.nMuon
    pt      = np.array(t.Muon_pt)          # note: this is an array
    phi     = np.array(t.Muon_phi)         # note: this is an array
    eta     = np.array(t.Muon_eta)         # note: this is an array 
    mass    = np.array(t.Muon_mass)        # note: this is an array
    isTight = np.array(t.Muon_tightId)     # note: this is an array
    charge  = np.array(t.Muon_charge)      # note: this is an array
    muons   = {
        'n':nMuon,
        'pt':pt,
        'phi':phi,
        'eta':eta,
        'isTight':isTight,
        'charge':charge,
    }
    ### YOUR CODE HERE###

In [ ]:
h_pT_z.Draw()
c.Update()
c.Draw()

In [ ]:
h_pTz_phi.Draw('COLZ')
c.Update()
c.Draw()

# Part 4d Improving fit quality

- Modify the fit to have a sum of a Gaussian and an exponent to describe the background under the peak 
- Does this improve the fit chi2?
- Can you think of other ways to improve the fit quality?

You may want to read the `TF1` documentation to think of which functions to use!

Here are some skeleton code for you to use:

In [36]:
## Define the sum of gaussian and exponential:
f2 = r.TF1('f2','gaus(0)+expo(3)',75,105)
print(f2.GetFormula().GetExpFormula()) 
f2.SetParameters(1,90,10,6,-0.02)

[p0]*exp(-0.5*((x-[p1])/[p2])*((x-[p1])/[p2]))+exp([p3]+[p4]*x)


In [ ]:
## ADD your fitting code here:

In [ ]:
h_dimuon_mass.Draw()
c.Update()
c.Draw()

In [ ]:
## Now check your fit quality. Does it improve?
chi2 = f2.GetChisquare()
NDF  = f2.GetNDF()
print('Checking fit quality:')
print("chi2 = %.3f\nNDF = %.3f\nchi2/NDF = %.3f"% (chi2,NDF, chi2/NDF))